In [1]:
# Import all the Dependencies
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
# pip uninstall numpy
# pip install numpy=1.14.0
# if error for multiple dll comes.!

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=5

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [5]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 1 2 0 0]


In [6]:
# plt.figure(figsize=(10, 10))
# for image_batch, labels_batch in dataset.take(1):
#     for i in range(12):
#         ax = plt.subplot(3, 4, i + 1)
#         plt.imshow(image_batch[i].numpy().astype("uint8"))
#         plt.title(class_names[labels_batch[i]])
#         plt.axis("off")

In [6]:
len(dataset)

68

In [7]:
train_size = 0.8
len(dataset)*train_size

54.400000000000006

In [8]:
train_ds = dataset.take(54)
len(train_ds)

54

In [9]:
test_ds = dataset.skip(54)
len(test_ds)

14

In [10]:
val_size=0.1
len(dataset)*val_size

6.800000000000001

In [11]:
val_ds = test_ds.take(6)
len(val_ds)

6

In [12]:
test_ds = test_ds.skip(6)
len(test_ds)

8

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [15]:
len(train_ds)

54

In [17]:
len(val_ds)

6

In [18]:
len(test_ds)

8

In [19]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [21]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [22]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [23]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [36]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [37]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50
)

Epoch 1/50
54/54 [==============================] - 103s 2s/step - loss: 0.9132 - accuracy: 0.4677 - val_loss: 0.9978 - val_accuracy: 0.4010
Epoch 2/50
54/54 [==============================] - 70s 1s/step - loss: 0.7154 - accuracy: 0.6567 - val_loss: 0.6230 - val_accuracy: 0.7292
Epoch 3/50
54/54 [==============================] - 73s 1s/step - loss: 0.5029 - accuracy: 0.7553 - val_loss: 0.4747 - val_accuracy: 0.7865
Epoch 4/50
54/54 [==============================] - 74s 1s/step - loss: 0.3196 - accuracy: 0.8727 - val_loss: 0.4484 - val_accuracy: 0.7812
Epoch 5/50
54/54 [==============================] - 73s 1s/step - loss: 0.2607 - accuracy: 0.8903 - val_loss: 0.2693 - val_accuracy: 0.8750
Epoch 6/50
54/54 [==============================] - 72s 1s/step - loss: 0.2356 - accuracy: 0.9061 - val_loss: 0.4210 - val_accuracy: 0.8333
Epoch 7/50
54/54 [==============================] - 73s 1s/step - loss: 0.2350 - accuracy: 0.9067 - val_loss: 0.2698 - val_accuracy: 0.8958
Epoch 8/50
54/54 [=

In [27]:
# from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
# pre = Precision()
# re = Recall()
# acc = BinaryAccuracy()
# for batch in test_ds.as_numpy_iterator():
#     X, y = batch
#     yhat = model.predict(X)
#     pre.update_state(y, yhat)
#     re.update_state(y, yhat)
#     acc.update_state(y, yhat)
    
# print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [38]:
scores = model.evaluate(test_ds)

8/8 [==============================] - 3s 373ms/step - loss: 0.1426 - accuracy: 0.9531


In [39]:
scores

[0.14256271719932556, 0.953125]

In [40]:
history

In [41]:
history.params

{'verbose': 1, 'epochs': 50, 'steps': 54}

In [42]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [43]:
type(history.history['loss'])

list

In [44]:
len(history.history['loss'])

50

In [45]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [46]:
import numpy as np
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
#     predicted_class = class_names[predictions[0]]
    confidence = round(100 * (np.max(predictions[0])), 2)
#     confidence = round(100 * (predictions[0]), 2)
    return predicted_class, confidence

In [47]:
for images, labels in test_ds.take(1):
    for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        print(f"Actual: {actual_class}, \nPredicted: {predicted_class}.\n Confidence: {confidence}%")
#         f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
#         plt.axis("off")

1/1 [==============================] - 0s 174ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 99.2%
1/1 [==============================] - 0s 31ms/step
Actual: Potato___Late_blight, 
Predicted: Potato___Late_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 34ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 99.99%
1/1 [==============================] - 0s 44ms/step
Actual: Potato___Late_blight, 
Predicted: Potato___Late_blight.
 Confidence: 100.0%
1/1 [==============================] - 0s 41ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 99.98%
1/1 [==============================] - 0s 41ms/step
Actual: Potato___Early_blight, 
Predicted: Potato___Early_blight.
 Confidence: 99.23%
1/1 [==============================] - 0s 38ms/step
Actual: Potato___Late_blight, 
Predicted: Potato___Late_blight.
 Confidence: 100.0%
1/1 [==============================] - 0

In [40]:
# import os
# model_version = 1
# model.save(f"../models/{model_version}")
# model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
# for creating new version above line
# if os.path.isfile('models/disease.h5') is False:
#     model.save('model/disease.h5')

In [48]:
import tensorflow as tf
print(tf. __version__)

2.11.0


In [49]:
MODEL = tf.keras.models.load_model('../saved_models/1')

In [50]:
2+2

4